VisuaDL中mIOU变化趋势或训练日志
（请在此处插入图片）断点续训图片
![](https://ai-studio-static-online.cdn.bcebos.com/768af9bb78f84f36891b72e50651c48b570c4174880640ceb675a4c4205db6d0)


infer/4346.png图片车道线分割效果图
（请在此处插入图片）
![](https://ai-studio-static-online.cdn.bcebos.com/689b50a70bb04d4c82d61206610f7165bac6a3c9c6e24f72a11b978fc0a4544c)


：[第十六届全国大学生智能车竞赛线上资格赛：车道线检测](https://aistudio.baidu.com/aistudio/competition/detail/68)比赛提交结果
（请在此处插入图片）
![](https://ai-studio-static-online.cdn.bcebos.com/415e83a45c564f329ed91db9e9e0e38313ec262835914118a70ab1817ab69ddb)


# 使用PaddleSeg进行车道线分割

## 解压数据集

In [ ]:
!unzip data/data82548/智能车数据集.zip

In [ ]:
#!git clone https://gitee.com/paddlepaddle/PaddleSeg.git

## [准备数据集](https://gitee.com/paddlepaddle/PaddleSeg/blob/release/v2.0/docs/data_prepare.md)
PaddleSeg目前支持CityScapes、ADE20K、Pascal VOC等数据集的加载，在加载数据集时，如若本地不存在对应数据，则会自动触发下载(除Cityscapes数据集)。
这里可以直接使用比赛提供的脚本

In [ ]:
%run make_list.py
!python make_list.py

In [ ]:
%set_env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


## 类别不均衡的处理

在图像分割任务中，经常出现类别分布不均匀的情况，例如：工业产品的瑕疵检测、道路提取及病变区域提取等。

针对这个问题，PaddleSeg主要提供了带权的softmax loss和Lovasz loss两种解决方案。

### [](https://gitee.com/paddlepaddle/PaddleSeg/blob/release/v0.6.0/docs/loss_select.md#weighted-softmax-loss)Weighted softmax loss

Weighted softmax loss是按类别设置不同权重的softmax loss。

通过设置`cfg.SOLVER.CROSS_ENTROPY_WEIGHT`参数进行使用。  
默认为None. 如果设置为'dynamic'，会根据每个batch中各个类别的数目，动态调整类别权重。 也可以设置一个静态权重(list的方式)，比如有3类，每个类别权重可以设置为\[0.1, 2.0, 0.9\]。
```
SOLVER:
    LR: 0.005
    LR_POLICY: "poly"
    OPTIMIZER: "sgd"
    NUM_EPOCHS: 40
    CROSS_ENTROPY_WEIGHT: [0.1, 2, 0.5, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2 ,2, 2] #每一个类别loss权重
```
### [](https://gitee.com/paddlepaddle/PaddleSeg/blob/release/v2.0/docs/lovasz_loss.md#lovasz-loss)Lovasz loss
Lovasz loss基于子模损失(submodular losses)的凸Lovasz扩展，对神经网络的mean IoU损失进行优化。Lovasz loss根据分割目标的类别数量可分为两种：lovasz hinge loss和lovasz softmax loss. 其中lovasz hinge loss适用于二分类问题，lovasz softmax loss适用于多分类问题。该工作发表在CVPR 2018上，可点击[参考文献](https://gitee.com/paddlepaddle/PaddleSeg/blob/release/v2.0/docs/lovasz_loss.md#%E5%8F%82%E8%80%83%E6%96%87%E7%8C%AE)查看具体原理。

需要注意的是，通常的直接训练方式不一定管用，PaddleSeg推荐的是另外2种训练方式：
- （1）与cross entropy loss或bce loss(binary cross-entropy loss)加权结合使用。
- （2）先使用cross entropy loss或bce loss进行训练，再使用lovasz softmax loss或lovasz hinge loss进行finetuning。
通过`coef`参数对不同loss进行权重配比，从而灵活地进行训练调参。

```python
loss:
  types:
    - type: MixedLoss
      losses:
        - type: CrossEntropyLoss
        - type: LovaszSoftmaxLoss
      coef: [0.8, 0.2]
    - type: MixedLoss
      losses:
        - type: CrossEntropyLoss
        - type: LovaszSoftmaxLoss
      coef: [0.8, 0.2]
  coef: [1, 0.4]

SOLVER:
    LR: 0.005
    LR_POLICY: "poly"
    OPTIMIZER: "sgd"
    NUM_EPOCHS: 40
    CROSS_ENTROPY_WEIGHT: dynamic
```

In [ ]:
%cd PaddleSeg

/home/aistudio/PaddleSeg


## 训练

In [ ]:
# 可以选择任意config文件重新训练
!python train.py \
       --config configs/ocrnet/ocrnet_hrnetw18_cityscapes_1024x512_160k_lovasz_softmax.yml \
       --do_eval \
       --use_vdl \
       --save_interval 750 \
       --save_dir output

In [ ]:
# 也可以在项目提供的6000轮训练结果上继续训练
!python train.py \
       --config configs/ocrnet/ocrnet_hrnetw18_cityscapes_1024x512_160k_lovasz_softmax.yml \
       --resume_model output/iter_12750 \
       --do_eval \
       --use_vdl \
       --save_interval 750 \
       --save_dir output

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2021-04-13 17:28:42 [INFO]	
------------Environment Information-------------
platform: Linux-4.13.0-36-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Cuda compilation tools, release 10.1, V10.1.243
cudnn: 7.6
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: Tesla V100-SXM2-16GB']
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddlePaddle: 2.0.1
OpenCV: 4.1.1
------------------------------------------------
2021-04-13 17:28:42 [INFO]	
---------------Config Information---------------
SOLVER:
  CROSS_ENTROPY_WEIGHT: dynamic
  LR: 0.005
  LR_POLICY: poly
  NUM_EPOCHS: 40
  OPTIMIZER: sgd
batch_size: 4
iters: 20000
learning_rate:
  decay:
    end_lr: 0.0
    power: 0.9
    

## 评估

In [ ]:
!python val.py \
       --config configs/ocrnet/ocrnet_hrnetw18_cityscapes_1024x512_160k_lovasz_softmax.yml \
       --model_path output/best_model/model.pdparams

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2021-04-13 20:20:05 [INFO]	
---------------Config Information---------------
SOLVER:
  CROSS_ENTROPY_WEIGHT: dynamic
  LR: 0.005
  LR_POLICY: poly
  NUM_EPOCHS: 40
  OPTIMIZER: sgd
batch_size: 4
iters: 20000
learning_rate:
  decay:
    end_lr: 0.0
    power: 0.9
    type: poly
  value: 0.0025
loss:
  coef:
  - 1
  - 0.4
  types:
  - coef:
    - 0.8
    - 0.2
    losses:
    - type: CrossEntropyLoss
    - type: LovaszSoftmaxLoss
    type: MixedLoss
  - coef:
    - 0.8
    - 0.2
    losses:
    - type: CrossEntropyLoss
    - type: LovaszSoftmaxLoss
    type: MixedLoss
model:
  backbone:
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/hrnet_w18_ssld.tar.gz
    type: HRNet_W18
  backbone_indices:
  - 0
  type: OCRNet
optimizer:
  momentum: 0.9
  type: sgd


## 预测

In [ ]:
!python predict.py \
       --config configs/ocrnet/ocrnet_hrnetw18_cityscapes_1024x512_160k_lovasz_softmax.yml \
       --model_path output/best_model/model.pdparams \
       --image_path ../infer/4346.png \
       --save_dir output/result

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2021-04-13 20:29:06 [INFO]	
---------------Config Information---------------
SOLVER:
  CROSS_ENTROPY_WEIGHT: dynamic
  LR: 0.005
  LR_POLICY: poly
  NUM_EPOCHS: 40
  OPTIMIZER: sgd
batch_size: 4
iters: 20000
learning_rate:
  decay:
    end_lr: 0.0
    power: 0.9
    type: poly
  value: 0.0025
loss:
  coef:
  - 1
  - 0.4
  types:
  - coef:
    - 0.8
    - 0.2
    losses:
    - type: CrossEntropyLoss
    - type: LovaszSoftmaxLoss
    type: MixedLoss
  - coef:
    - 0.8
    - 0.2
    losses:
    - type: CrossEntropyLoss
    - type: LovaszSoftmaxLoss
    type: MixedLoss
model:
  backbone:
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/hrnet_w18_ssld.tar.gz
    type: HRNet_W18
  backbone_indices:
  - 0
  type: OCRNet
optimizer:
  momentum: 0.9
  type: sgd


## 导出静态图模型

In [ ]:
!python export.py \
       --config configs/ocrnet/ocrnet_hrnetw18_cityscapes_1024x512_160k_lovasz_softmax.yml \
       --model_path output/best_model/model.pdparams

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
W0413 20:30:27.097597 10098 device_context.cc:362] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.0, Runtime API Version: 10.1
W0413 20:30:27.102846 10098 device_context.cc:372] device: 0, cuDNN Version: 7.6.
2021-04-13 20:30:31 [INFO]	Loading pretrained model from https://bj.bcebos.com/paddleseg/dygraph/hrnet_w18_ssld.tar.gz
2021-04-13 20:30:31,020 - INFO - Lock 140131666467024 acquired on /home/aistudio/.paddleseg/tmp/hrnet_w18_ssld
2021-04-13 20:30:31,021 - INFO - Lock 140131666467024 released on /home/aistudio/.paddleseg/tmp/hrnet_w18_ssld
2021-04-13 20:30:32 [INFO]	There are 1525/1525 variables loaded into HRNet.
2021-04-13 20:30:32 [INFO]	Loaded trained params of model successfully.
/opt/conda/envs/python35-paddle120-env/lib/

In [ ]:
%cd /home/aistudio/

/home/aistudio


In [ ]:
# 进行推理，其中--model_path后跟自己训练好的模型文件路径，一般选择bast_model或iter后最大值的参数文件
!python PaddleSeg/predict.py --config PaddleSeg/configs/ocrnet/ocrnet_hrnetw18_cityscapes_1024x512_160k_lovasz_softmax.yml --model_path PaddleSeg/output/best_model/model.pdparams --image_path infer --save_dir output/result

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2021-04-13 20:38:16 [INFO]	
---------------Config Information---------------
SOLVER:
  CROSS_ENTROPY_WEIGHT: dynamic
  LR: 0.005
  LR_POLICY: poly
  NUM_EPOCHS: 40
  OPTIMIZER: sgd
batch_size: 4
iters: 20000
learning_rate:
  decay:
    end_lr: 0.0
    power: 0.9
    type: poly
  value: 0.0025
loss:
  coef:
  - 1
  - 0.4
  types:
  - coef:
    - 0.8
    - 0.2
    losses:
    - type: CrossEntropyLoss
    - type: LovaszSoftmaxLoss
    type: MixedLoss
  - coef:
    - 0.8
    - 0.2
    losses:
    - type: CrossEntropyLoss
    - type: LovaszSoftmaxLoss
    type: MixedLoss
model:
  backbone:
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/hrnet_w18_ssld.tar.gz
    type: HRNet_W18
  backbone_indices:
  - 0
  type: OCRNet
optimizer:
  momentum: 0.9
  type: sgd


In [ ]:
%cd /home/aistudio/output/result/pseudo_color_prediction
!zip -r -o /home/aistudio/predict.zip ./
%cd /home/aistudio